In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas_bokeh as pb
# from bokeh.plotting import figure, show
# from bokeh.io import output_notebook, export_png, output_file
# from bokeh.layouts import row, gridplot
# from bokeh.models import ColumnDataSource
import numpy as np
import random
import datetime
# import seaborn as sns; sns.set()
# %matplotlib inline
# pb.output_notebook()
# output_notebook()

In [2]:
starttime = datetime.datetime.now()

In [3]:
# 已知规则
# 1. 客户来源_原始 如果该字段为空，则自动填充为 展厅

In [4]:
filename = 'potential.202101.pkl'
monstr = filename.split('.')[1]

In [5]:
databookALL = pd.read_pickle("Data/AudiPotentialCustomer/History/%s" % filename, compression='bz2')

In [6]:
# databook = databookALL.drop(['车型', '大区', '客户来源更新', '月份'], axis=1)
databook = databookALL.copy()

In [7]:
databookALL['潜客量'].sum()

338134

In [8]:
# setting.xlsx 格式
# 每个sheet的名字会被用来作为字段映射时的字典名，所以最好用英文，比如现在setting里的sheet名就是"cartype","bigarea"
# 同时，sheet的内容是一个字段的映射关系，比如 大区名称->大区
#      A    B
#  1  东北区 A1
# 这样读取的时候就可以按照下面去读了
tExcel = pd.read_excel("Data/AudiPotentialCustomer/settingALL.xlsx", sheet_name=None)

In [9]:
# 这里settingHash就是未来做dataframe.map()用的
settingHash = {}
for keys in tExcel:
    settingHash[keys] = dict(tExcel[keys].values)

In [10]:
tExcel.keys()

odict_keys(['city', 'cname', 'bigarea', 'cartype'])

In [11]:
# 客户来源_原始 中 所有NaN值 都会赋值为展厅
databook['客户来源更新'] = databook['客户来源_原始'].fillna('展厅')

databook['车型new'] = databook['分组名称'].map(settingHash['cartype'])

In [12]:
# (2021-02-08) 保存原始车型信息，所以不用考虑空值
# map的方式举例，m没用，但是建议将其保留，备以后查
# 当有新的车型进来时，可以在这里看到。
# m = pd.DataFrame({'a':databook['分组名称'].head(200), 'b':databook['分组名称'].head(200)})
# m['b'] = m['a'].map(settingHash['cartype'])
# m.drop_duplicates(['a','b'],  'first')

In [13]:
def transArr(df):
    # 将 [[A4L, 10], [A6, 5]], 这种类型的改成10个A4L，5个A6
    # [A4L, A4L, A4L, A4L, A4L, A4L, A4L, A4L, A4L, A4L, A6, A6, A6, A6, A6]
    # 方便后面按照随机取数出来，这样取出来应该就是这种分布
    tmp = []
    for idx, row in df.iterrows():
        tmp.extend([row['车型new']] * row['潜客量'])
    return tmp

In [14]:
# 清空dataMiss时，要用这种办法
# dataMiss.drop(dataMiss.index, inplace=True)

In [15]:
def genNewData(dCode, cSource, nowPOS):
    # dCode 经销商代码
    # cSource 客户来源更新
    # 进来后再做循环
    # 这里需要注意，dataMiss到底有哪些列
    tmpArr = transArr(dataCOMPUTed[(dataCOMPUTed['经销商代码']==dCode)&(dataCOMPUTed['客户来源更新']==cSource)&(dataCOMPUTed['潜客量']>0)][['车型new', '潜客量']])
    for idx, row in dataBYday[(dataBYday['经销商代码']==dCode)&(dataBYday['客户来源更新']==cSource)].iterrows():
        toDel = []
        for aIDX in random.sample(range(len(tmpArr)), int(row['潜客量'])):
            dataMiss.loc[nowPOS] = [
                pd.to_datetime(row['日期']),
                row['大区名称'],
                row['小区名称'],
                row['经销商代码'],
                1,
                row['客户来源更新'],
                tmpArr[aIDX]
            ]
            nowPOS +=1 
            toDel.append(aIDX)
        for i in sorted(toDel, reverse=True):
            del(tmpArr[i])
    return nowPOS

In [16]:
def tempDF(lvl, df):
    tmp = df[[lvl, '客户来源更新', '车型new', '潜客量']].copy()
    tmp = tmp.groupby([lvl, '客户来源更新', '车型new'])['潜客量'].sum().reset_index()
    tmp['%s汇总' % lvl] = tmp.groupby([lvl, '客户来源更新'])['潜客量'].transform('sum')
    tmp['车型占比'] = tmp['潜客量']/tmp['%s汇总' % lvl]
    tmp.drop(['潜客量', '%s汇总' % lvl], axis=1, inplace=True)
    return tmp

In [17]:
def batDraw(lvl):
    graphics = []
    vals = list(getIDX(lvl, dataUSEstd))
    df_o = tempDF(lvl, dataUSEstd)
    df_c = tempDF(lvl, dataEsti)

    for t in ["展厅", "线上"]:
        tmp = []
        for i in vals:
            tmp.append(drawGraph(
                df_o[(df_o[lvl]==i)&(df_o['客户来源更新']==t)][['车型new', '车型占比']],
                df_c[(df_c[lvl]==i)&(df_c['客户来源更新']==t)][['车型new', '车型占比']],
                '%s %s' % (i, t)
            ))
        graphics.append(tmp)

    return graphics

In [18]:
def drawGraph(se1, se2, title='aa'):
    tmpdf = se1.merge(se2, on=['车型new'], suffixes=['_origin', '_fix'])
    tmpdf.set_index(['车型new'], inplace=True)
    m = tmpdf.plot_bokeh(kind='barh', legend='top_right', title=title, show_figure=False, zooming=False)
    return m

In [19]:
def resetType(ser):
    if pd.isnull(ser['分组名称']):
        return np.nan
    else:
        return ser['车型']

In [20]:
def covTypes(df):
    for col in df.columns:
        if col == '潜客量':
            df[col] = df[col].astype('int32')
        elif col== '日期':
            continue
        else:
            df[col] = df[col].astype('category')

In [21]:
def getIDX(cName, df, c=6):
    # 在df[cName]的最高30%里，随机取c个作为输出
    # 用于将来的分布验证
    tmp = df[[cName, '潜客量']].groupby([cName])['潜客量'].sum().reset_index()
    tmplist = df[cName].unique()
    if int(len(tmplist)*0.3) <= c:
        tmpidx = tmp.sort_values(by='潜客量', ascending=False).index.values
    else:
        tmpidx = tmp.sort_values(by='潜客量', ascending=False)[:int(len(tmplist)*0.3)].index.values
    return np.array(tmp[cName].iloc[random.sample(list(tmpidx), c)])

In [22]:
def balanceCom(rows):
    # 根据差值选择计算方式
    if rows['差值'] < 0:
        if rows['已进位'] and rows['顺序']+rows['差值']<=0:
            return rows['预计算潜客2'] - 1
    elif rows['差值'] > 0:
        if (not rows['已进位']) and rows['顺序'] - rows['差值'] <= 0:
            return rows['预计算潜客2'] + 1
    return rows['预计算潜客2']

In [23]:
def getColumns(col, vals,ty):
    # 获取画图的列
    oriData = dataUSEstd[(dataUSEstd[col]==vals)&(dataUSEstd['客户来源更新']==ty)][['车型new', '车型占比']]
    comData = dataEsti[(dataEsti[col]==vals)&(dataEsti['客户来源更新']==ty)][['车型new', '车型占比']]
    # oriData = statRES(col, dataUSEstd)[]
    return oriData.merge(comData, on='车型new', suffixes=['_origin', '_fix'])

In [24]:
# 这里确实要删掉 分组名称，因为分组和车型new是多对一，如果不删，将来merge回来时，分组名称会为NaN
dataAvailable = databook[databook['车型new'].notna()]
dataAvailable = dataAvailable.drop(['分组名称', '客户来源_原始'], axis=1)

In [25]:
dataAvailable['潜客量'].sum(), databook[databook['车型new'].isna()]['潜客量'].sum()

(303777, 34357)

In [26]:
dataMiss = pd.DataFrame(columns=list(dataAvailable.columns))

In [27]:
# dataUSE = databook[['客户来源更新', '大区名称', '小区名称', '城市', '经销商代码', '车型new', '潜客量']].copy()
dataUSE = databook[['大区名称', '小区名称', '经销商代码', '潜客量', '客户来源更新', '车型new']]

In [28]:
# cityLIST = dataUSE['城市'].unique()
area1LIST = dataUSE['大区名称'].unique()
area2LIST = dataUSE['小区名称'].unique()
dealerLIST = dataUSE['经销商代码'].unique()
print(len(area1LIST), len(area2LIST), len(dealerLIST))#, len(cityLIST)

6 56 578


In [29]:
monthDays = len(databook['日期'].unique())

In [30]:
dataBYday = databook[databook['车型new'].isna()].groupby(['日期', '大区名称', '小区名称', '经销商代码', '客户来源更新'])['潜客量'].sum().reset_index()

In [31]:
# 计算 经销商 客户来源 和车型 的group by sum，去掉日期来计算概率
# sum(经销商、客户来源、车型)/monthDays 这就是某经销商在某来源下车型出现的概率
# dealerDISTbd.groupby(["经销商代码", "客户来源更新", "车型new", "日期"])["潜客量"].sum().reset_index()
# 按日的做起来太麻烦所以不用这个了

In [32]:
dataUSEstd = dataUSE[dataUSE['车型new'].notna()]
dataUSE2b = dataUSE[dataUSE['车型new'].isna()]

In [33]:
# 汇总车型数据
dataUSEstd = dataUSEstd.groupby(['大区名称', '小区名称', '经销商代码', '客户来源更新', '车型new'])['潜客量'].sum().reset_index()

In [34]:
# (2021-02-08)创建一个基于车型的城市分布，这里改成 大区分布 
# 因为后续会有部分经销商没有对应 "客户来源更新" 的分布，比如某些经销商没有线上的已有数据作为基准，这时就用城市的替换
# dataCITYstd = dataUSEstd.groupby(['城市', '客户来源更新', '车型new'])['潜客量'].sum().reset_index()
dataCITYstd = dataUSEstd.groupby(['大区名称', '车型new'])['潜客量'].sum().reset_index()

In [35]:
# 可能存在来源更新没有的情况，处理起来太复杂，直接用城市了
# dataCITYstd['城市汇总'] = dataCITYstd.groupby(['城市', '客户来源更新'])['潜客量'].transform('sum')
dataCITYstd['大区汇总'] = dataCITYstd.groupby(['大区名称'])['潜客量'].transform('sum')
dataCITYstd['车型占比'] = dataCITYstd['潜客量']/ dataCITYstd['大区汇总']

In [36]:
dataUSEstd['dealer汇总'] = dataUSEstd.groupby(['大区名称', '小区名称', '经销商代码', '客户来源更新'])['潜客量'].transform('sum')
dataUSEstd['车型占比'] = dataUSEstd['潜客量']/dataUSEstd['dealer汇总']
dataUSEstd['日均概率'] = dataUSEstd['潜客量']/monthDays

In [37]:
dataUSE2b = dataUSE2b.groupby(['大区名称', '小区名称', '经销商代码', '客户来源更新'])['潜客量'].sum().reset_index()

In [38]:
datatmp = dataUSE2b.copy()

In [39]:
datatmp = datatmp.merge(dataUSEstd[["经销商代码", "客户来源更新", "车型new", "车型占比"]], on=["经销商代码", "客户来源更新"], how='left')

In [40]:
noneSTD = datatmp[datatmp['车型占比'].isna()]

In [41]:
datatmp = datatmp[datatmp['车型占比'].notna()]

In [42]:
noneSTD.drop(['车型new', '车型占比'], axis=1, inplace=True)

In [43]:
noneSTD = noneSTD.merge(dataCITYstd[['大区名称', '车型new', '车型占比']], on=['大区名称'])

In [44]:
datatmp = pd.concat([datatmp, noneSTD], ignore_index=True)

In [45]:
# 预计算潜客1 是未四舍五入的值， 预计算潜客2 是四舍五入后的值
datatmp['预计算潜客'] = datatmp["潜客量"] * datatmp["车型占比"]
datatmp['预计算潜客1'] = datatmp['预计算潜客'].map(int)
datatmp['预计算潜客2'] = datatmp['预计算潜客'].apply(lambda x: int(x+0.5))
datatmp['进位可能'] = datatmp['预计算潜客'] - datatmp['预计算潜客1']

In [46]:
# 这里计算出来的 差值 就是四舍五入后的差异，需要用这个差异去修改 datatmp
# 里的 预计算潜客2 中不合适的部分。
# 并在 datatmp 中插入新的列 调整潜客
# 所以需要利用 进位可能，进行多退少补：
#     如果需要退n个（差值为负），则寻找 进位可能 > 0.5的数据里最小的n个各去掉 1
#     如果需要补n个（差值为正），则寻找 进位可能 < 0.5的数据里最大的n个各增加 1
# 这部分需要使用函数计算，感觉还比较麻烦
# 这里直接做一个 进位可能 相关的函数，主要是插入 顺序 字段，值是序数：
#   大于 0.5 的 升序，比如：
#     进位可能: [0.51, 0.66, 0.501, 0.7]
#        顺序: [1, 2, 0, 3]
#   小于 0.5 的 降序
##########################################

In [47]:
# 1. 给 datatemp 增加一列，区分 0.5
datatmp['已进位'] = datatmp['进位可能'] >= 0.5

In [48]:
# 都使用的升序，处理时要注意
# 先处理 已进位 为 True的
datatmp['顺序'] = 0
l = datatmp['顺序'] + datatmp[datatmp['已进位']==True].groupby(['经销商代码', '客户来源更新', '已进位'])['进位可能'].rank(ascending=True, method="first").astype('int')
r = datatmp['顺序'] + datatmp[datatmp['已进位']==False].groupby(['经销商代码', '客户来源更新', '已进位'])['进位可能'].rank(ascending=False, method="first").astype('int')
datatmp['顺序'] = l.fillna(0) + r.fillna(0)

In [49]:
# 计算经销商分来源的预计算潜客2的汇总
dataUSEpre = datatmp.groupby(['大区名称', '小区名称', '经销商代码', '客户来源更新'])['预计算潜客2'].sum().reset_index()

In [50]:
#dataUSEadj = datatmp.groupby(['大区名称', '小区名称', '城市', '经销商代码', '客户来源更新'])['预计算潜客2'].sum().reset_index()
#dataUSEadj = dataUSE2b.head(200).merge(dataUSEpre[['经销商代码', '客户来源更新', '预计算潜客2']], on=['经销商代码', '客户来源更新'])
dataUSEadj = dataUSE2b.merge(dataUSEpre[['经销商代码', '客户来源更新', '预计算潜客2']], on=['经销商代码', '客户来源更新'])
dataUSEadj['差值'] = dataUSEadj['潜客量'] - dataUSEadj['预计算潜客2']

In [51]:
# 将 差值 合并进 datatmp
datatmp = datatmp.merge(dataUSEadj[['经销商代码', '客户来源更新', '差值']], on=['经销商代码', '客户来源更新'])

In [52]:
datatmp['预计算真值'] = datatmp.apply(balanceCom, axis=1)

In [53]:
# 这里开始合并模拟结果和真实结果

In [54]:
dataCOMPUTed = datatmp[['大区名称', '小区名称', '经销商代码', '客户来源更新','车型new', '预计算真值']].copy()

In [55]:
dataCOMPUTed.rename(columns={'预计算真值': '潜客量'}, inplace=True)
dataCOMPUTed.head()

,大区名称,小区名称,经销商代码,客户来源更新,车型new,潜客量
0,东北区,吉林小区,SA15016,线上,A3 Lim,0
1,东北区,吉林小区,SA15016,线上,A3 SpB,1
2,东北区,吉林小区,SA15016,线上,A4L,4
3,东北区,吉林小区,SA15016,线上,A6L,6
4,东北区,吉林小区,SA15016,线上,Q2L,1


In [56]:
dataEsti = dataCOMPUTed.append(dataUSEstd[['大区名称', '小区名称', '经销商代码', '客户来源更新','车型new', '潜客量']])

In [57]:
dataEsti = dataEsti.groupby(['大区名称', '小区名称', '经销商代码', '客户来源更新','车型new'])['潜客量'].sum().reset_index()

In [58]:
# 添加车型站比
dataEsti['dealer汇总'] = dataEsti.groupby(['大区名称', '小区名称', '经销商代码', '客户来源更新'])['潜客量'].transform('sum')
dataEsti['车型占比'] = dataEsti['潜客量']/dataEsti['dealer汇总']
dataEsti['日均概率'] = dataEsti['潜客量']/monthDays

In [59]:
#### 以下是之前处理的结果

In [60]:
def summarySTAT():
    totalPotentials = databook['潜客量'].sum()
    # dealPotentialsbyDay = databook.groupby(['经销商代码', '日期'])['潜客量'].sum().reset_index()
    totalDealPotentials = databook.groupby(['经销商代码'])['潜客量'].sum().reset_index()
    totalPotentialsFIX = dataEsti['潜客量'].sum()
    # dealPotentialsbyDayFIX = dataEsti.groupby(['经销商代码', '日期'])['潜客量'].sum().reset_index()
    totalDealPotentialsFIX = dataEsti.groupby(['经销商代码'])['潜客量'].sum().reset_index()
    # dealDiffbyDay = dealPotentialsbyDay.merge(dealPotentialsbyDayFIX, on=['经销商代码', '日期'], suffixes=['_origin', '_fix'])
    dealDiff = totalDealPotentials.merge(totalDealPotentialsFIX, on=['经销商代码'], suffixes=['_origin', '_fix'])
    # dealDiffbyDay['差值'] = dealDiffbyDay['潜客量_origin'] - dealDiffbyDay['潜客量_fix']
    dealDiff['差值'] = dealDiff['潜客量_origin'] - dealDiff['潜客量_fix']
    print(dealDiff['差值'].value_counts())
    return totalPotentials - totalPotentialsFIX, dealDiff['差值'].value_counts()

In [61]:
dealerUnique = dataCOMPUTed[['大区名称', '小区名称', '经销商代码', '客户来源更新']].drop_duplicates()
dealerUnique = dealerUnique.reset_index(drop=True)[['经销商代码', '客户来源更新']]

In [62]:
start = 0
for idx, row in dealerUnique.iterrows():
    print(idx, row['经销商代码'], row['客户来源更新'])
    start = genNewData(row['经销商代码'], row['客户来源更新'], start)
    print(start)

0 SA15016 线上
14
1 SA22001 线上
33
2 SA22002 线上
61
3 SA22003 线上
81
4 SA22008 线上
104
5 SA22009 线上
108
6 SA22012 线上
165
7 SA22015 线上
168
8 SA22016 线上
183
9 SA15009 线上
188
10 SA21002 线上
401
11 SA21005 线上
421
12 SA21006 线上
444
13 SA21013 线上
550
14 SA21015 线上
677
15 SA21016 线上
795
16 SA21018 线上
798
17 SA21020 线上
802
18 SA21024 线上
831
19 SA21026 线上
900
20 SA21027 线上
1010
21 SA21007 线上
1034
22 SA21009 线上
1053
23 SA21010 线上
1148
24 SA21011 线上
1180
25 SA21012 线上
1248
26 SA21014 线上
1316
27 SA21021 线上
1349
28 SA21022 线上
1399
29 SA21023 线上
1435
30 SA21025 线上
1437
31 SA37010 线上
1579
32 SA37014 线上
1651
33 SA37017 线上
1756
34 SA37018 线上
1770
35 SA37021 线上
1872
36 SA37037 线上
2084
37 SA37041 线上
2253
38 SA37043 线上
2323
39 SA37052 线上
2331
40 SA37053 线上
2336
41 SA37517 线上
2356
42 SA37005 线上
2368
43 SA37007 线上
2419
44 SA37013 线上
2576
45 SA37023 线上
2662
46 SA37025 线上
2684
47 SA37026 线上
2753
48 SA37028 线上
2786
49 SA37042 展厅
2787
50 SA37042 线上
2847
51 SA37055 线上
2884
52 SA37505 线上
2893
53 SA37525 线上
2894
54 SA376

In [63]:
dataMiss.head(1)

,日期,大区名称,小区名称,经销商代码,潜客量,客户来源更新,车型new
0,2021-01-09,东北区,吉林小区,SA15016,1,线上,Q5L


In [64]:
# a1 = dataMiss.groupby(['大区名称', '小区名称', '城市', '经销商代码', '客户来源更新', '车型new'])['潜客量'].sum().reset_index()

# a2 = a1.merge(dataCOMPUTed, on=['大区名称', '小区名称', '城市', '经销商代码', '客户来源更新', '车型new'], suffixes=['_o', '_f'])

In [65]:
# 图表所需的层次
# (2021-02-08)去掉了城市维度
# graphLVL = ["大区名称", "小区名称", "城市", "经销商代码"]
graphLVL = ["大区名称", "小区名称", "经销商代码"]

In [66]:
pics = []
for pic in graphLVL:
    pics.extend(batDraw(pic))
html = pb.plot_grid(pics, plot_width=300, return_html=True)

In [67]:
with open("Results/AudiPotentialCustomer/History/DistributionCheck.%s.html" % monstr, 'w') as f:
    f.write(html)

In [68]:
# 轮盘赌算法，应该用不着了，直接用估计值去填坑
# import random
# maxnum = 10000
# pDIS = [0.046414, 0.050633, 0.147679, 0.147679, 0.270042, 0.008439, 0.004219, 0.033755, 0.025316, 0.07173, 0.004219, 0.008439, 0.181435]
# pDISsum = []
# for i in range(len(pDIS)):
#     if pDISsum:
#         pDISsum.append(pDISsum[-1] + pDIS[i])
#     else:
#         pDISsum.append(pDIS[i])
# result = [0] * len(pDIS)
# for i in range(maxnum):
#     rNUM = random.random()
#     for n, m in enumerate(pDISsum):
#         if rNUM < m:
#             result[n] += 1
#             break
# print(list(map(lambda x: '%.3f' % x, pDIS)))      
# print(list(map(lambda x: '%.3f' % (x/maxnum), result)))

In [69]:
dataMiss['潜客量'].sum(), dataAvailable['潜客量'].sum()

(34357, 303777)

In [70]:
databookNew = dataAvailable.append(dataMiss, ignore_index=True)

In [71]:
databookNew.head(1)

,日期,大区名称,小区名称,经销商代码,潜客量,客户来源更新,车型new
0,2021-01-01,北部区,北京1区,SA11003,1,线上,A3 SpB


In [72]:
# 这是一个总数检查，databookNew = databookALL
databookNew['潜客量'].sum(),databookALL['潜客量'].sum(), dataAvailable['潜客量'].sum(), dataMiss['潜客量'].sum()

(338134, 338134, 303777, 34357)

In [73]:
# databookNew = databookNew.groupby(['月份', '日期', '大区名称', '大区', '小区名称', '经销商代码', '经销商名称', '城市', '客户来源更新', '车型new'])['潜客量'].sum().reset_index()
databookNew = databookNew.groupby(['日期', '大区名称', '小区名称', '经销商代码', '客户来源更新', '车型new'])['潜客量'].sum().reset_index()

In [74]:
# 这里做 城市、经销商名称、大区 的生成
# databookNew['大区'] = databook['大区名称'].map(settingHash['bigarea'])
databookNew['大区'] = databookNew['大区名称'].map(settingHash['bigarea'])
databookNew['月份'] = databookNew['日期'].map(lambda x: str(x.month)+"月")
databookNew['经销商名称'] = databookNew['经销商代码'].map(settingHash['cname'])
databookNew['城市'] = databookNew['经销商代码'].map(settingHash['city'])

In [75]:
# databookNew 做类型转换，减少体积
covTypes(databookNew)

In [76]:
# 存两个文件
# 1. pkl压缩版，万一以后要操作就不必从excel中读取了
# 2. excel文件，分发给相关工作人员
databookNew.to_pickle('Results/AudiPotentialCustomer/History/potential.mani.%s.pkl' % monstr, compression='bz2')
databookNew.to_excel('Results/potential.%s.xlsx' % monstr, encoding='utf8', header=True, index=False)

In [77]:
endtime = datetime.datetime.now()

In [78]:
(endtime - starttime).seconds

267